# Calculate sidewalk width
Using centerline extraction (also called skeleton line, axis line, or medial line extraction)

In [ ]:
# Add project src to path.
import set_path

import numpy as np
import pathlib
import pandas as pd
import geopandas as gpd
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
import shapely.ops as so

import upcp.utils.bgt_utils as bgt_utils
import upcp.utils.las_utils as las_utils

import upc_sw.poly_utils as poly_utils

In [ ]:
import warnings  # temporary, to supress deprecationwarnings from shapely
warnings.filterwarnings('ignore')

In [ ]:
# Optional, for parallel processing of geodataframes.
import dask_geopandas as dgpd
from dask.diagnostics import ProgressBar
pbar = ProgressBar()
pbar.register()

In [ ]:
# Paths
pc_data_folder = '../datasets/pointclouds/'
out_folder = '../datasets/output/'

# A CRS tells Python how those coordinates relate to places on the Earth. Rijksdriehoek = epsg:28992
CRS = 'epsg:28992' #local crs

# Whether to merge sidewalks before segmentation and width computation
merge_sidewalks = True

# Tolerance for centerline simplification
simplify_tolerance = 0.2

# Min area size of a sidewalk polygon in sqm for which width will be computed
min_area_size = 1

# Max linestring length in meters
max_line_length = 5

# Resolution (in m) for min and avg width computation
width_resolution = 1

# Precision (in decimals) for min and avg width computation
width_precision = 1

## Read the sidewalk and obstacle data

In [ ]:
# Read sidewalk with obstacle data
obstacle_file = f'{out_folder}/sidewalks_with_obstacles.gpkg'
df = gpd.read_file(obstacle_file, geometry='geometry', crs=CRS)

if merge_sidewalks:
    # Merge sidewalk polygons
    df = gpd.GeoDataFrame(geometry=gpd.GeoSeries([geom for geom in df.unary_union.geoms]), crs=CRS)
else:
    # Explode MultiPolygons into their parts
    df = df.explode(index_parts=False)

# Ignore sidewalk polygons that are too small
df = df[df.area > min_area_size]

## Calculate the centerlines

In [ ]:
def my_func(row):
    return poly_utils.get_centerlines(row.geometry)

### Single thread

In [ ]:
# Single thread, can be slow for big dataframes.

# if you get an error here, make sure you use tqdm>=4.61.2
df['centerlines'] = df.progress_apply(my_func, axis=1)
df = df.set_geometry('centerlines')

### Parallel

In [ ]:
# Alternative: parallel processing for big dataframes.

# TODO: sensible (or variable) setting for npartitions
ddf = dgpd.from_geopandas(df, npartitions=24)

In [ ]:
# TODO: we could specify all operations for the entire notebook here so everything gets parallelized in one go
ddf['centerlines'] = ddf.apply(my_func, axis=1, meta=(object))

In [ ]:
# Perform the ectual computations and convert back to a single dataframe
df = ddf.compute()

In [ ]:
df = df.set_geometry('centerlines')

## Remove short line ends and dead-ends

In [ ]:
df['centerlines'] = df['centerlines'].progress_apply(so.linemerge)

In [ ]:
df['centerlines'] = df['centerlines'].progress_apply(
                        lambda x: poly_utils.remove_short_lines(x, max_line_length))

## Get sidewalk widths

In [ ]:
df['centerlines'] = df['centerlines'].progress_apply(
                        lambda row: row.simplify(simplify_tolerance, preserve_topology=True))

In [ ]:
df['segments'] = df['centerlines'].progress_apply(poly_utils.get_segments)

In [ ]:
df[['avg_width', 'min_width']] = df.progress_apply(
                        lambda row: poly_utils.get_avg_width(row.geometry, row.segments,
                                                             width_resolution, width_precision),
                        axis=1)

In [ ]:
# TODO: keep sidewalk ID along with segments for back-reference
segment_gdf = pd.concat([gpd.GeoDataFrame({'geometry': row.segments,
                                           'avg_width': row.avg_width,
                                           'min_width': row.min_width})
                         for _, row in df.iterrows()])
segment_gdf.set_crs(crs=CRS, inplace=True);

## Check coverage of point cloud data on sidewalks

In [ ]:
all_tiles = las_utils.get_tilecodes_from_folder(f'{pc_data_folder}run1/')
all_tiles_poly = so.unary_union([poly_utils.tilecode_to_poly(tile) for tile in all_tiles])

segment_gdf['pc_coverage'] = segment_gdf.intersects(all_tiles_poly)

## Store output

In [ ]:
pathlib.Path(out_folder).mkdir(parents=True, exist_ok=True)

segments_file = f'{out_folder}sidewalk_segments.gpkg'
segment_gdf.to_file(segments_file, driver='GPKG')

## Plot results

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt


tilecodes = ['2386_9702']

plot_shape = so.unary_union([poly_utils.tilecode_to_poly(tilecode) for tilecode in tilecodes])
(x_min, y_min, x_max, y_max) = plot_shape.bounds
df_plot = df[df.intersects(plot_shape)]

fig, ax = plt.subplots(1, figsize=(6,6))

df_plot.set_geometry('geometry').plot(ax=ax, cmap='tab10')

ax.set_xlim([x_min, x_max])
ax.set_ylim([y_min, y_max])

plt.show()